# Treinamento CIS - 4º Período (Redes Neurais)


Este notebook implementa uma rede neural do zero para detectar fraudes em transações de cartão de crédito. Utilizamos técnicas de balanceamento de dados e normalização para melhorar o desempenho do modelo.

Importações

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from scipy.special import expit

In [2]:
file_path = 'creditcard.csv'
credit_card = pd.read_csv(file_path, sep=',', on_bad_lines='skip', low_memory=False)
print(credit_card.info())
print(credit_card.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

## Definir modelos da rede neural

In [3]:
# Definição da camada da rede neural
class NeuralLayer:
    def __init__(self, input_size, neuron_count):
        self.weights = np.random.randn(input_size, neuron_count) * 0.01
        self.biases = np.zeros((1, neuron_count))

    def forward(self, inputs):
        self.input = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, gradient, learning_rate):
        self.dweights = np.dot(self.input.T, gradient) / self.input.shape[0]
        self.dbiases = np.sum(gradient, axis=0, keepdims=True) / self.input.shape[0]
        self.weights -= learning_rate * self.dweights
        self.biases -= learning_rate * self.dbiases
        self.dinput = np.dot(gradient, self.weights.T)

# Funções de ativação
class ReLUActivation:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

    def backward(self, gradient):
        self.dinput = gradient.copy()
        self.dinput[self.output <= 0] = 0

class SigmoidActivation:
    def forward(self, inputs):
        self.output = expit(inputs)
        
    def backward(self, gradient):
        self.dinput = gradient * (self.output * (1 - self.output))

# Função de perda
class BinaryCrossEntropyLoss:
    def calculate(self, predictions, targets):
        predictions = np.clip(predictions, 1e-7, 1 - 1e-7)
        return -np.mean(targets * np.log(predictions) + (1 - targets) * np.log(1 - predictions))

    def backward(self, predictions, targets):
        predictions = np.clip(predictions, 1e-7, 1 - 1e-7)
        return (predictions - targets) / (predictions * (1 - predictions))

## Carregar e Preparar os Dados
Nesta seção, carregamos o dataset e separamos as features das labels. Também normalizamos as features para garantir que todas tenham a mesma escala.


In [4]:
# Carregar e preparar os dados
def load_and_prepare_data(file_path):
    data_frame = pd.read_csv(file_path)
    features = data_frame.drop('Class', axis=1)
    labels = data_frame['Class']
    return features, labels

# Normalizar as variáveis independentes
def normalize_features(features):
    scaler = StandardScaler()
    return scaler.fit_transform(features)

# Aplicar o SMOTE para balancear as classes
def balance_classes(features, labels, strategy=0.3):
    smote = SMOTE(sampling_strategy=strategy, random_state=42)
    return smote.fit_resample(features, labels)

## Definição e Treinamento do Modelo
Aqui definimos a estrutura da rede neural e implementamos o processo de treinamento. A rede possui três camadas: duas camadas ocultas com ReLU e uma camada de saída com Sigmoid.

In [5]:

# Função de treinamento e avaliação
def train_and_evaluate_model(X_train, y_train, X_val, y_val, learning_rate, epochs, batch_size):
    layer1 = NeuralLayer(X_train.shape[1], 124)
    activation1 = ReLUActivation()
    layer2 = NeuralLayer(124, 2)
    activation2 = ReLUActivation()
    layer3 = NeuralLayer(2, 1)
    activation3 = SigmoidActivation()
    loss_function = BinaryCrossEntropyLoss()

    for epoch in range(epochs):
        for start in range(0, X_train.shape[0], batch_size):
            batch_X = X_train[start:start + batch_size]
            batch_y = y_train.iloc[start:start + batch_size].values.reshape(-1, 1)

            # Forward pass
            layer1.forward(batch_X)
            activation1.forward(layer1.output)
            layer2.forward(activation1.output)
            activation2.forward(layer2.output)
            layer3.forward(activation2.output)
            activation3.forward(layer3.output)

            # Backward pass
            gradient = loss_function.backward(activation3.output, batch_y)
            activation3.backward(gradient)
            layer3.backward(activation3.dinput, learning_rate)
            activation2.backward(layer3.dinput)
            layer2.backward(activation2.dinput, learning_rate)
            activation1.backward(layer2.dinput)
            layer1.backward(activation1.dinput, learning_rate)

        # Avaliação no conjunto de validação
        layer1.forward(X_val)
        activation1.forward(layer1.output)
        layer2.forward(activation1.output)
        activation2.forward(layer2.output)
        layer3.forward(activation2.output)
        activation3.forward(layer3.output)

        val_loss = loss_function.calculate(activation3.output, y_val.values.reshape(-1, 1))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {val_loss:.4f}")

    return layer1, activation1, layer2, activation2, layer3, activation3



## Função de treinamento

In [6]:
# Configuração de hiperparâmetros
def main(learning_rates,epochs,batch_size,best_model,best_f1_score):
    features, labels = load_and_prepare_data('creditcard.csv')
    normalized_features = normalize_features(features)
    X_train, X_test, y_train, y_test = train_test_split(normalized_features, labels, test_size=0.5, random_state=42)
    X_resampled, y_resampled = balance_classes(normalized_features, labels)

    X_train_resampled, X_val_resampled, y_train_resampled, y_val_resampled = train_test_split(
        X_resampled, y_resampled, test_size=0.25, random_state=42)


    for learning_rate in learning_rates:
        print(f"\nTreinando com taxa de aprendizado = {learning_rate}")
        model = train_and_evaluate_model(X_train_resampled, y_train_resampled, X_val_resampled, y_val_resampled, learning_rate, epochs, batch_size)

        layer1, activation1, layer2, activation2, layer3, activation3 = model

        # Avaliação no conjunto de validação
        layer1.forward(X_val_resampled)
        activation1.forward(layer1.output)
        layer2.forward(activation1.output)
        activation2.forward(layer2.output)
        layer3.forward(activation2.output)
        activation3.forward(layer3.output)

        y_pred_val = (activation3.output >= 0.5).astype(int)
        accuracy_val = accuracy_score(y_val_resampled, y_pred_val)
        precision_val = precision_score(y_val_resampled, y_pred_val)
        recall_val = recall_score(y_val_resampled, y_pred_val)
        f1_val = f1_score(y_val_resampled, y_pred_val)

        print(f"Validação: Acurácia = {accuracy_val:.4f}")
        print(f"Validação: Precisão = {precision_val:.4f}")
        print(f"Validação: Recall = {recall_val:.4f}")
        print(f"Validação: F1 = {f1_val:.4f}")


        # Selecionar o melhor modelo
        if f1_val > best_f1_score:
            best_f1_score = f1_val
            best_model = model

    # Previsão no conjunto de teste
    layer1, activation1, layer2, activation2, layer3, activation3 = best_model

    layer1.forward(X_test)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    activation2.forward(layer2.output)
    layer3.forward(activation2.output)
    activation3.forward(layer3.output)

    y_pred_test = (activation3.output >= 0.5).astype(int)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test)
    recall_test = recall_score(y_test, y_pred_test)
    f1_test = f1_score(y_test, y_pred_test)

    # Resultados finais
    print("\nResultados finais:")
    print(f"Acurácia = {accuracy_test:.4f}")
    print(f"Precisão = {precision_test:.4f}")
    print(f"Recall = {recall_test:.4f}")
    print(f"F1 = {f1_test:.4f}")




Definir parâmetros de treinamento e treinar o modelo.

In [7]:
learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2]
epochs = 300
batch_size = 32
best_model = None
best_f1_score = 0

main(learning_rates,epochs,batch_size,best_model,best_f1_score)


Treinando com taxa de aprendizado = 0.001
Epoch 0: Loss = 0.5394
Epoch 100: Loss = 0.0078
Epoch 200: Loss = 0.0038
Validação: Acurácia = 0.9994
Validação: Precisão = 0.9976
Validação: Recall = 1.0000
Validação: F1 = 0.9988

Treinando com taxa de aprendizado = 0.005
Epoch 0: Loss = 0.4325
Epoch 100: Loss = 0.0030
Epoch 200: Loss = 0.0028
Validação: Acurácia = 0.9996
Validação: Precisão = 0.9981
Validação: Recall = 1.0000
Validação: F1 = 0.9991

Treinando com taxa de aprendizado = 0.01
Epoch 0: Loss = 0.1039
Epoch 100: Loss = 0.0034
Epoch 200: Loss = 0.0033
Validação: Acurácia = 0.9996
Validação: Precisão = 0.9984
Validação: Recall = 1.0000
Validação: F1 = 0.9992

Treinando com taxa de aprendizado = 0.05
Epoch 0: Loss = 0.0238
Epoch 100: Loss = 0.0018
Epoch 200: Loss = 0.0023
Validação: Acurácia = 0.9995
Validação: Precisão = 0.9977
Validação: Recall = 1.0000
Validação: F1 = 0.9988

Treinando com taxa de aprendizado = 0.1
Epoch 0: Loss = 0.0129
Epoch 100: Loss = 0.0034
Epoch 200: Loss =

/home/caefleury/Documents/unb/trainee-cis-ieee/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validação: Acurácia = 0.7687
Validação: Precisão = 0.0000
Validação: Recall = 0.0000
Validação: F1 = 0.0000

Resultados finais:
Acurácia = 0.9997
Precisão = 0.8601
Recall = 1.0000
F1 = 0.9248


## O código implementa uma rede neural do zero para detecção de fraudes em transações de cartão de crédito.

### Pré-processamento dos Dados:

Carregamento do dataset e separação das features e labels.
Normalização das features para garantir que todas tenham a mesma escala.
Uso do SMOTE para balancear as classes, aumentando a quantidade de exemplos da classe minoritária.

### Estrutura da Rede Neural:

A rede possui três camadas: duas camadas ocultas com a função de ativação ReLU e uma camada de saída com a função de ativação Sigmoid.
A função de perda utilizada é a Binary Cross-Entropy.

### Treinamento e Avaliação:

O modelo é treinado com diferentes taxas de aprendizado, variando de 0.001 a 0.2.
Durante o treinamento, a perda é monitorada a cada 100 épocas.

O desempenho do modelo é avaliado usando métricas como acurácia, precisão, recall e F1-score no conjunto de validação.


## Análise dos Resultados

### Taxas de Aprendizado Baixas a Moderadas (0.001 a 0.1):

As taxas de aprendizado de 0.001 a 0.1 resultaram em um bom desempenho, com acurácia e F1-score próximos de 1.0.

A perda diminuiu significativamente ao longo das épocas, indicando que o modelo estava aprendendo bem.

A precisão, recall e F1-score foram todos muito altos, sugerindo que o modelo estava equilibrado em termos de predição de classes positivas e negativas.

### Taxas de Aprendizado Altas (0.15 e 0.2):

As taxas de aprendizado de 0.15 e 0.2 resultaram em um desempenho significativamente pior.

A perda aumentou drasticamente, indicando que o modelo estava instável e não convergindo adequadamente.

A precisão e o F1-score caíram drasticamente, especialmente para a taxa de aprendizado de 0.2, onde a precisão foi definida como 0 devido à falta de amostras previstas corretamente.

### Resultados Finais:

O melhor desempenho foi observado com taxas de aprendizado entre 0.005 e 0.01, onde o modelo alcançou uma acurácia de 0.9997 e um F1-score de 0.9248 no conjunto de teste.

Isso sugere que essas taxas de aprendizado proporcionaram um bom equilíbrio entre velocidade de convergência e estabilidade do modelo.
